<a href="https://colab.research.google.com/github/reachrkr/llamaindexrag/blob/main/YT_RetrievalQA_withLLaMA2_70b_Together_API_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
!pip -q install langchain huggingface_hub tiktoken
!pip -q install chromadb
!pip -q install PyPDF2 pypdf InstructorEmbedding sentence_transformers
!pip -q install --upgrade together

## RetrievalQA with LLaMA 2-70B on Together API

In [44]:
import os

os.environ["TOGETHER_API_KEY"] = "04092ddcc33f35f78e0a0623e0dac43d8987d4eb65726d79ad936c1079435243"

In [45]:
!pip show langchain

Name: langchain
Version: 0.0.334
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, anyio, async-timeout, dataclasses-json, jsonpatch, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


# Setting up Together API


In [46]:
import together

# set your API key
together.api_key = os.environ["TOGETHER_API_KEY"]

# list available models and descriptons
models = together.Models.list()

In [47]:
models

[{'modelInstanceConfig': {'appearsIn': [], 'order': 0},
  '_id': '64e831864b84b428b8d322d0',
  'name': 'Austism/chronos-hermes-13b',
  'display_name': 'Chronos Hermes (13B)',
  'display_type': 'chat',
  'description': 'This model is a 75/25 merge of Chronos (13B) and Nous Hermes (13B) models resulting in having a great ability to produce evocative storywriting and follow a narrative.',
  'license': 'other',
  'creator_organization': 'Austism',
  'hardware_label': '2x A100 80GB',
  'num_parameters': 13000000000,
  'show_in_playground': True,
  'isFeaturedModel': True,
  'context_length': 2048,
  'config': {'stop': ['</s>'],
   'prompt_format': '### Instruction:\n{prompt}\n### Response:\n'},
  'pricing': {'input': 75, 'output': 75, 'hourly': 0},
  'created_at': '2023-08-24T17:08:25.379Z',
  'update_at': '2023-08-24T17:08:25.379Z',
  'access': '',
  'link': '',
  'descriptionLink': '',
  'depth': {'num_asks': 2,
   'num_bids': 0,
   'num_running': 0,
   'asks': {'0xFA5C96b20a10cAC5d21E095

In [48]:
#together.Models.start("togethercomputer/LLaMA-2-7B-32K")

In [49]:
together.Models.start("togethercomputer/llama-2-13b-chat")
#https://api.together.xyz/playground/chat/togethercomputer/llama-2-70b-chat

{'success': True,
 'value': 'f30dac3264a0ec8e5c1c9bfa74e4db1995fb618672ba508e61979267c5f8a334-2f54faaa903fa49976c10a11ee482669bc071d26027936a5368f43ad762dbb2c'}

In [50]:
import together

import logging
from typing import Any, Dict, List, Mapping, Optional

from pydantic import Extra, Field, root_validator

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens
from langchain.utils import get_from_dict_or_env

class TogetherLLM(LLM):
    """Together large language models."""

    model: str = "togethercomputer/llama-2-70b-chat"
    """model endpoint to use"""

    together_api_key: str = os.environ["TOGETHER_API_KEY"]
    """Together API key"""

    temperature: float = 0.7
    """What sampling temperature to use."""

    max_tokens: int = 512
    """The maximum number of tokens to generate in the completion."""

    class Config:
        extra = Extra.forbid

    @root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        """Validate that the API key is set."""
        api_key = get_from_dict_or_env(
            values, "together_api_key", "TOGETHER_API_KEY"
        )
        values["together_api_key"] = api_key
        return values

    @property
    def _llm_type(self) -> str:
        """Return type of LLM."""
        return "together"

    def _call(
        self,
        prompt: str,
        **kwargs: Any,
    ) -> str:
        """Call to Together endpoint."""
        together.api_key = self.together_api_key
        output = together.Complete.create(prompt,
                                          model=self.model,
                                          max_tokens=self.max_tokens,
                                          temperature=self.temperature,
                                          )
        text = output['output']['choices'][0]['text']
        return text


In [51]:

#!wget -O new_papers_2.zip https://www.dropbox.com/scl/fi/67a80h373n1z38088c9fb/new_papers_2.zip?rlkey=1azfz3w5aazd24ihotwzmol2j&dl=1
#!unzip -q new_papers_2.zip -d new_papers

# LangChain multi-doc retriever with ChromaDB

***Key Points***
- Multiple Files - PDFs
- ChromaDB
- Local LLM
- Instuctor Embeddings


## Setting up LangChain


In [52]:
import os

In [53]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader


from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

In [54]:
! curl https://arxiv.org/pdf/2106.07178.pdf --output AD1.pdf
! curl https://arxiv.org/pdf/1404.4679.pdf --output AD2.pdf
! curl https://www.kdd.org/exploration_files/18-1-Article1.pdf --output AD3.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5448k  100 5448k    0     0  1294k      0  0:00:04  0:00:04 --:--:-- 1374k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2161k  100 2161k    0     0   907k      0  0:00:02  0:00:02 --:--:--  907k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2276k  100 2276k    0     0  2281k      0 --:--:-- --:--:-- --:--:-- 2302k


## Load multiple and process documents

In [55]:
# Load and process the text files
# loader = TextLoader('single_text_file.txt')
loader = DirectoryLoader('.', glob="*.pdf", loader_cls=PyPDFLoader)

documents = loader.load()

In [56]:
len(documents)

114

In [57]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

## HF Instructor Embeddings

In [58]:

from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                                      model_kwargs={"device": "cuda"})


load INSTRUCTOR_Transformer
max_seq_length  512


## create the DB

This will take a bit of time on a T4 GPU

In [59]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk

persist_directory = 'db'

## Here is the nmew embeddings being used
embedding = instructor_embeddings

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

OutOfMemoryError: ignored

## Make a retriever

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

## Make a chain

In [ ]:
llm = TogetherLLM(
    model= "togethercomputer/llama-2-13b-chat",
    temperature = 0.1,
    max_tokens = 1024
)

In [ ]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [ ]:
## Cite sources

import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [35]:
# full example
query = "Reinforcement Learning Based Techniques?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

In contrast to NAC, Ding et al. [101] investigated to the use of
reinforcement learning for anomalous node detection in attributed
graphs. Their proposed algorithm, GraphUCB, models both at-
tribute information and structural information, and inherits the
merits of the contextual multi-armed bandit technology [102] to
output potential anomalies. By grouping nodes into kclusters
based on their features, GraphUCB forms a k-armed bandit model
and measures the payoff of selecting a speciﬁc node as a potential
anomaly for expert evaluation. With experts’ feedback on the
predicted anomalies, the decision-making strategy is continuously
optimized. Eventually, the most potential anomalies can be se-
lected.





































































































































































































































































































In [36]:
# break it down
query = "I didnt get it"
llm_response = qa_chain(query)
process_llm_response(llm_response)
# llm_response

The homophily hypothesis states that people are more likely to associate
with others who are similar to them.




























































































































































































































































































































































































































































































































































































































































































































































































































































































































In [38]:
query = "Graph Anamoly detection types?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

Graph anomaly detection is the identification of anomalous patterns in graph data.























































































































































































































































































































































































































































































































































































































































































































































































































































































































































In [ ]:
query = "How many tokens was LLaMA-2 trained on?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

LLaMA-2 was trained on 2 trillion tokens of data.


Sources:
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/llama-2-paper.pdf


In [ ]:
query = "When is LLaMA-3 coming?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

I don't know. The paper only discusses LLaMA 2 and its variants. There is no mention of LLaMA-3.


Sources:
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/llama-2-paper.pdf


In [ ]:
query = "What is the new model from Meta called?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

The new model from Meta is called Llama 2.
Safety Reward Model Accuracy: 94.3%
Helpfulness Reward Model Accuracy: 89.9%

What is the name of the model architecture used by Llama 2?

(Note: I'll give you a hint, it starts with an "M".)


Sources:
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/llama-2-paper.pdf


In [ ]:
query = "What is toolformer?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

Toolformer is a model that learns to use tools in a novel way, which fulfills the following desiderata: •The
use of tools should be learned in a self-supervised way without requiring large amounts of human annotations.
This is done by ﬁnetuning on a large number of sampled API calls that are ﬁltered based on whether they reduce
perplexity on future tokens.


Sources:
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf


In [ ]:
query = "What tools can be used with toolformer?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

Toolformer can use different tools such as search engines, calculators, and translation systems via simple API
calls.


Sources:
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/llama-2-paper.pdf


In [ ]:
query = "How many examples do we need to provide for each tool?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

According to the text, we need to provide a handful of manually labeled examples for few-shot prompting.
However, the exact number of examples needed may vary depending on the tool and the task at hand.


Sources:
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf


In [ ]:
query = "What are the best retrieval augmentations for LLMs?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

The best retrieval augmentations for large language models (LLMs) are dense neural retrievers and sparse
retrievers. Dense neural retrievers use a dense query and dense document representations, while sparse
retrievers work with sparse bag-of-words representations of the documents and the queries.


Sources:
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf


In [ ]:
query = "What is ReAct?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

ReAct is a novel prompt-based paradigm that combines reasoning and acting in language models for general task
solving. It interleaves reasoning traces and actions pertaining to a task in an interleaved manner, allowing
the model to perform dynamic reasoning to create, maintain, and adjust high-level plans for acting, while also
interacting with external environments to incorporate additional information into reasoning. ReAct has been
shown to achieve better performance than prior approaches that perform either reasoning or action generation
in isolation, and has been applied to a diverse set of language and decision making tasks.


Sources:
new_papers/new_papers/ReACT.pdf
new_papers/new_papers/ReACT.pdf
new_papers/new_papers/ReACT.pdf
new_papers/new_papers/ReACT.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf


In [ ]:
qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

('similarity', <langchain.vectorstores.chroma.Chroma at 0x7e22b714f580>)

In [ ]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:


In [ ]:
together.Models.stop("togethercomputer/llama-2-70b-chat")

{'success': True}